In [1]:
import numpy as numpy
import os
import json


with open("tokens.json", "r", encoding="utf-8") as f:
    private_keys = json.load(f)


folder_id = private_keys['yandex_cloud_dir_identifier']
api_key = private_keys['yandex_cloud_key']


### get texts

In [2]:
# в качестве маленького датасета возьмем кусок старого конспекта базового курса по машинке
titles = [
    "Machine Learning problem statement. Regression, Classification, examples.",
    "How to measure quality in classification: accuracy, balanced accuracy, precision, recall, f1-score, ROC-AUC, multiclass extensions",
    "How to measure quality in regression: MSE, MAE, R2.",
    "Maximum likelihood estimation, how is it related to regression and classification",
    "Naive bayesian classifier, how does it work",
    "K-nearest neighbours classifier, how does it work",
    "Linear regression. Problem statement for the MSE loss function case. Analytical solution. Gauss-Markov theorem. Gradient approach in linear regression.",
    "Regularization in linear models: L1 и L2, their properties. Probabilistic interpretation.",
    "Logistic regression. Equivalence of MLE approach and logistic loss minimization.",
    "Multiclass classification. One-vs-one, one-vs-all, their properties.",
    "Support vector machine. Optimization problem for SVM. Kernel trick. Kernel properties.",
    "Principal component analysis. Relations to SVD. Eckart-Young theorem. How to apply PCA in practice.",
    "Train, validation and test stages of model development. Overfitting problem, ways to detect it.",
    "Validation strategies. Cross validation. Data leaks.",
    "Bias-variance tradeoff.",
    "Decision tree construction procedure.",
    "Information criteria. Entropy criteria, Giny impurity.",
    "Ensembling methods. Bootstrap. Bagging.",
    "Random Forest, Random subspace method.",
    "Boosting and gradient boosting. Main idea, gradient derivation.",
    "Matrix calculus and matrix derivatives. How to get the derivative of matrix/dot product",
    "Backpropagation, chain rule."
]

texts = []

texts_dir = "D:/Desktop/9сем/LLM/RAG_pipeline_for_LLM_course/5sem_ml/section/"

for file in sorted(os.listdir(texts_dir), key=lambda x: int(x.split(".")[0])):
    with open(texts_dir + file, "r", encoding="utf-8") as f:
        texts.append(f.read())

In [3]:
texts_with_titles = [{
        "text": text[:6000],
        "title": title
    } for title, text in zip(titles, texts)
]

### get embedder

In [4]:
from yandex_chain import YandexEmbeddings

embeddings = YandexEmbeddings(
    folder_id=folder_id, 
    api_key=api_key
)

vec = embeddings.embed_query("Hello, world!")
len(vec)

256

### load into database

In [5]:
!rm -rf db

from langchain_chroma import Chroma

db = Chroma(
    embedding_function=embeddings, 
    persist_directory='./db'
)

for d in texts_with_titles:
    db.add_texts(
        [d["title"]], 
        metadatas=[{
            "title": d["title"],
            "text": d["text"]
        }]
    )

In [6]:
# retriever test:
query = 'Что такое линейная регрессия? Какие метрики используются в задачах регрессии?'

retriever = db.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 5}
)

retrieved = retriever.invoke(query)
retrieved


[Document(id='03ce01f5-e0ca-4625-b558-3750b3b579ac', metadata={'text': 'Напомним, что аналитическое решение для метода наименьших квадратов выглядит следующим образом:\n\n\\[\n\\widehat{w} = (X^T X)^{-1} X^T Y\n\\]\n\nВидно, что нам приходится искать матрицу, обратную к $X^T X$. Однако, что делать, если эта матрица вырождена (или близка к вырожденной)? \n\nСначала подумаем, что означает вырожденность матрицы $X^T X$. Это означает, что в этой матрице (а значит, и в матрице $X$) есть линейно зависимые столбцы (признаки). В таком случае столбец весов не может быть определён однозначно, так как будет фиксирована только сумма весов этих параметров, а сами параметры могут определяться континуальным количеством способов.\n\nЕсли же матрица $X^T X$ близка к вырожденной, то решение получается \\textit{нестабильным} (то есть при добавлении небольшого шума в данные вектор весов меняется сильно).\n\nЧтобы решить эту проблему, нам хочется ограничить свободу выбора этих коэффициентов, наложить допол

In [7]:
[elem.metadata["title"] for elem in retrieved]

['Regularization in linear models: L1 и L2, their properties. Probabilistic interpretation.',
 'Logistic regression. Equivalence of MLE approach and logistic loss minimization.',
 'Linear regression. Problem statement for the MSE loss function case. Analytical solution. Gauss-Markov theorem. Gradient approach in linear regression.',
 'Maximum likelihood estimation, how is it related to regression and classification',
 'Random Forest, Random subspace method.']

### add llm and pipeline via langchain graph

In [8]:
import langchain.chains
import langchain.prompts
from yandex_chain import YandexLLM, YandexGPTModel
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph

model = YandexLLM(
    folder_id=folder_id,
    api_key=api_key,
    model=YandexGPTModel.Pro
)

prompt = langchain.prompts.PromptTemplate(
    template = '''
        Посмотри на предложенные фрагменты конспекта и ответь на вопрос. Старайся использовать информацию из предложенного текста. Отвечай кратко, без дополнительных пояснений.
        Контекст:
        #####
        {context}
        #####
        Вопрос:
        #####
        {question}''',
    input_variables=["context", "question"]
)

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


def retrieve(state: State):
    retrieved_docs = retriever.invoke(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.metadata["text"] for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = model.invoke(messages)
    return {"answer": response}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [9]:
response = graph.invoke({"question": "Что такое линейная регрессия? Какие метрики используются в задачах регрессии?"})

In [12]:
print(response["answer"])

Линейная регрессия — это модель, использующая линейную функцию для предсказания значений на основе вектора признаков.

В задачах регрессии используются следующие метрики:
* MSE (среднеквадратичная ошибка);
* MAE (средняя абсолютная ошибка).
